In [1]:
!jupyter nbextension install --user --py pythreejs
!jupyter nbextension enable --user --py pythreejs

Installing /home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/pythreejs/static -> jupyter-threejs
Up to date: /home/kartik/.local/share/jupyter/nbextensions/jupyter-threejs/three.js
Up to date: /home/kartik/.local/share/jupyter/nbextensions/jupyter-threejs/extension.js
Up to date: /home/kartik/.local/share/jupyter/nbextensions/jupyter-threejs/index.js.map
Up to date: /home/kartik/.local/share/jupyter/nbextensions/jupyter-threejs/index.js
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable pythreejs --user --py
    
Enabling notebook extension jupyter-threejs/extension...
      - Validating: OK


In [2]:
import data_utils
%matplotlib notebook
#%matplotlib inline
import os
import sys
import math
import h5py
import argparse
import importlib
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import scipy.spatial as spatial
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

from matplotlib import colors as mcolors
from sklearn.cluster import DBSCAN
from matplotlib import animation
# import pptk
import numpy as np
import colorsys
from datetime import datetime

import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from pyntcloud import PyntCloud
from argoverse.map_representation.map_api import ArgoverseMap

from tqdm import tqdm_notebook as tqdm
import object3d

import pandas as pd

from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
from glob import glob


In [13]:
def createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color="yellow"):
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C3,C4,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C4,C8,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C6,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C2,C6,C7,C3,C2]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C3,C7,C8,C4,C3]
        })
    return bounding_box


def getCorners_argo(height,width,length,x,y,z,rotMat,rotation=True):

    corners = np.array([[-length / 2, -length / 2, length / 2, length / 2, -length / 2, -length / 2, length / 2, length / 2],
                        [width / 2, -width / 2, -width / 2, width / 2, width / 2, -width / 2, -width / 2, width / 2],
                        [-height/2, -height/2, -height/2, -height/2, height/2, height/2,  height/2, height/2]])
    if rotation:
        cornersPos = (np.dot(rotMat,corners)+np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    else:
        cornersPos = (corners + np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    
    return list(corner1),list(corner2),list(corner3),list(corner4),list(corner5),list(corner6),list(corner7),list(corner8)

lines = []
lines.append(
    {
        "color":'red', #x
        "vertices":[[5,0,0],[0,0,0],[0,0,0]]
    })
lines.append(
    {
        "color":'green', #y
        "vertices":[[0,0,0],[0,5,0],[0,0,0]]
    })
lines.append(
    {
        "color":'blue', #z
        "vertices":[[0,0,0],[0,0,0],[0,0,5]]
    })

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--filelist',  '-t', help='Path to training set ground truth (.txt)')
parser.add_argument('--filelist_val', '-v', help='Path to validation set ground truth (.txt)')
parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
parser.add_argument('--save_folder', '-s', help='Path to folder for saving check points and summary')
parser.add_argument('--model', '-m', help='Model to use')
parser.add_argument('--setting', '-x', help='Setting to use')
parser.add_argument('--epochs', help='Number of training epochs (default defined in setting)', type=int)
parser.add_argument('--batch_size', help='Batch size (default defined in setting)', type=int)
parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')
args = parser.parse_args([])

# if not args.no_timestamp_folder:
#     time_string = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
#     root_folder = os.path.join(args.save_folder, '%s_%s_%s_%d' % (args.model, args.setting, time_string, os.getpid()))
# else:
#     root_folder = args.save_folder
# if not os.path.exists(root_folder):
#     os.makedirs(root_folder)
print(args)

cwd = os.getcwd()
dataset_dir  = os.path.join(cwd, "data", "KITTI")
filelist = os.path.join(dataset_dir, "ImageSets")
train_data = os.path.join(filelist, "train.txt")

args = {
        'repeat_num' : 1,
        'setting' : "kitti3d_x8_2048_fps",
        'save_ply' : False,
        'model' : "pointcnn_seg",
        'max_point_num' : 10000,
        'filelist' : train_data
}


Namespace(batch_size=None, epochs=None, filelist=None, filelist_val=None, load_ckpt=None, log='log.txt', model=None, no_code_backup=False, no_timestamp_folder=False, save_folder=None, setting=None)


In [5]:
model = importlib.import_module(args['model'])
setting_path = os.path.join(cwd, args['model'])
sys.path.append(setting_path)
setting = importlib.import_module(args['setting'])

num_epochs = setting.num_epochs
batch_size = setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
label_weights_list = setting.label_weights
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val

### Data prepare for model

In [3]:
import argparse
import os 
import numpy as np
import glob 
import datetime
import time 
import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from pyntcloud import PyntCloud
from argoverse.map_representation.map_api import ArgoverseMap

parser = argparse.ArgumentParser()
parser.add_argument('--folder', '-f', help='Path to data folder')
parser.add_argument('--max_point_num', '-m', help='Max point number of each sample', type=int, default=8192)
parser.add_argument('--block_size', '-b', help='Block size', type=float, default=5.0)
parser.add_argument('--grid_size', '-g', help='Grid size', type=float, default=0.1)
parser.add_argument('--save_ply', '-s', help='Convert .pts to .ply', action='store_true')

args = parser.parse_args([])
print(args)

cwd = os.getcwd()

root = args.folder if args.folder else os.path.join(cwd, "data", "Argo") +"/"
save_h5_root =  os.path.join(cwd, "data", "Argo_h5")

if args.save_ply:
    data_center = np.zeros((batch_size, max_point_num, 3))
    
folders = [os.path.join(root, folder) for folder in ['train', 'val', 'test']]

Namespace(block_size=5.0, folder=None, grid_size=0.1, max_point_num=8192, save_ply=False)


In [4]:
folders

['/home/kartik/DL_model/PointCNN/data/Argo/train',
 '/home/kartik/DL_model/PointCNN/data/Argo/val',
 '/home/kartik/DL_model/PointCNN/data/Argo/test']

In [5]:
split = 'train'
root_dir = root
is_test = (split == 'test')
lidar_pathlist = []
label_pathlist = []
actual_idx_list = []
logidx_to_count_map= {}
log_to_count_map= {}

print("____________SPLIT IS : {} ______________".format(split))
if split == 'train':
    imageset_dir = os.path.join(root_dir,split)
    splitname = lambda x: [x[len(imageset_dir+"/"):-4].split("/")[0], x[len(imageset_dir+"/"):-4].split("/")[2].split("_")[1]]
    data_loader = ArgoverseTrackingLoader(os.path.join(root_dir,split))
    log_list = data_loader.log_list
    path_count = 0
    for log_id, log in enumerate(log_list):
        lidar_lst = data_loader.get(log).lidar_list
        lidar_pathlist.extend(lidar_lst)
        label_pathlist.extend(data_loader.get(log).label_list)
        actual_idx_list.extend([splitname(each) for each in lidar_lst])
        logidx_to_count_map[log_id] = np.arange(path_count, path_count + len(lidar_lst))
        log_to_count_map[log] = np.arange(path_count, path_count + len(lidar_lst))
        path_count+=len(lidar_lst)
    assert len(lidar_pathlist) == len(label_pathlist)

elif split == 'test':
    imageset_dir = os.path.join(root_dir,split)
    splitname = lambda x: [x[len(imageset_dir+"/"):-4].split("/")[0], x[len(imageset_dir+"/"):-4].split("/")[2].split("_")[1]]
    print("______________image set dir_________", imageset_dir)
    data_loader = ArgoverseTrackingLoader(os.path.join(root_dir,split))
    log_list = data_loader.log_list
    path_count = 0
    for log in log_list:
        lidar_pathlist.extend(data_loader.get(log).lidar_list)
        actual_idx_list.extend([splitname(each) for each in lidar_lst])
        logidx_to_count_map[log_id] = np.arange(path_count, path_count + len(lidar_lst))
        log_to_count_map[log] = np.arange(path_count, path_count + len(lidar_lst))
        path_count+=len(lidar_lst)
    print("The lidar list len is : ",len(self.lidar_pathlist))
    label_pathlist = None
    
    
am = ArgoverseMap()
calib_file = data_loader.calib_filename
print("sample from lidar pathsL ",lidar_pathlist[0])

num_sample = len(lidar_pathlist)
image_idx_list = np.arange(num_sample)
print("image list sample is: ", actual_idx_list[0])

if len(lidar_pathlist) > len(actual_idx_list):
    print("There is length difference between lidar and actual files : ", lidar_pathlist, " ",actual_idx_list )
else:
    print("len of list is same \n")

argo_to_kitti = np.array([[6.927964e-03, -9.999722e-01, -2.757829e-03],
                               [-1.162982e-03, 2.749836e-03, -9.999955e-01],
                               [9.999753e-01, 6.931141e-03, -1.143899e-03]])

ground_removal = True

____________SPLIT IS : train ______________


sample from lidar pathsL  /home/kartik/DL_model/PointCNN/data/Argo/train/3d20ae25-5b29-320d-8bae-f03e9dc177b9/lidar/PC_315975023020283000.ply
image list sample is:  ['3d20ae25-5b29-320d-8bae-f03e9dc177b9', '315975023020283000']
len of list is same 



In [78]:
import object3d
import json
from functools import reduce

def get_objects_from_label(label_file):
    # Opens a label file, and passes the object to Object3d object, Read the json GT labels
    
    f = open(label_file)
    label_data = json.load(f) 
    objects = [object3d.Object3d(data) for data in label_data]
    return objects

def filter_pointcloud(bbox, pointcloud):
    theta = bbox.ry #["angle"]
    transformed_pointcloud = homogeneous_transformation(pointcloud[:, :3], bbox.pos[:3], -theta)#["center"]
    if bbox.l > bbox.w:
        length = bbox.l
        width = bbox.w
    else:
        length = bbox.w
        width = bbox.l
            
#     indices = np.intersect1d(np.where(np.abs(transformed_pointcloud[:,0]) <= width/2)[0], 
#                              np.where(np.abs(transformed_pointcloud[:,1]) <= length/2)[0])
    
    indices = reduce(np.intersect1d, (np.where(np.abs(transformed_pointcloud[:,0]) <= width/2)[0], 
                            np.where(np.abs(transformed_pointcloud[:,1]) <= length/2)[0],
                            np.where(np.abs(transformed_pointcloud[:,2]) <= bbox.h/2)[0]))
    return indices, pointcloud[indices,:], transformed_pointcloud, bbox

def homogeneous_transformation(points, translation, theta):
    return (points[:, :3] - translation).dot(rotation_matrix_2D(theta).T)

def rotation_matrix_2D(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0],
                     [0,0,1]])


In [75]:
def data_preprocessing(data_input ,labels,  max_point_num):
    batch_size = 2048
    block_size = 1000
    grid_size = 0.25
        
    assert len(labels) == data_input.shape[0]
#     data = np.zeros((batch_size, max_point_num, 4))
    data = np.zeros((batch_size, max_point_num, 3))
    data_num = np.zeros((batch_size), dtype=np.int32)
    label = np.zeros((batch_size), dtype=np.int32)
    label_seg = np.zeros((batch_size, max_point_num), dtype=np.int32)
    indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)
    
    xyzi = data_input[:, :4]
    
    
    indices_for_prediction = np.arange(xyzi.shape[0]) #(xyzi[:,0] >= -5 ).nonzero()[0]
    #print("indices_for_prediction", indices_for_prediction)
    # Filter point only in front on of ego-sensors
    xyzif =xyzi #= xyzi[xyzi[:,0] >= -5 ] 
    
    all_label_pred = np.zeros((xyzi.shape[0]),dtype=int)
    label_length = xyzif.shape[0]
    xyz =xyzif[:,0:3]
    
            
    xyz_min = np.amin(xyz, axis=0, keepdims=True)
    xyz_max = np.amax(xyz, axis=0, keepdims=True)
    block_size = (2 * (xyz_max[0, 0] - xyz_min[0, 0]), 2 * (xyz_max[0, 1] - xyz_min[0, 1]) ,  2 * (xyz_max[0, -1] - xyz_min[0, -1]))
    
    xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)

    #print('{}-Collecting points belong to each block...'.format(datetime.now(), xyzrcof.shape[0]))
    blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True,
                                                                return_counts=True, axis=0)
    block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
    #print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

    block_to_block_idx_map = dict()
    for block_idx in range(blocks.shape[0]):
        block = (blocks[block_idx][0], blocks[block_idx][1])
        block_to_block_idx_map[(block[0], block[1])] = block_idx

    # merge small blocks into one of their big neighbors
    block_point_count_threshold = max_point_num / 3
    #print("block_point_count_threshold",block_point_count_threshold)
    nbr_block_offsets = [(0, 1), (1, 0), (0, -1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)]
    block_merge_count = 0
    for block_idx in range(blocks.shape[0]):
        if block_point_counts[block_idx] >= block_point_count_threshold:
            #print(block_idx, block_point_counts[block_idx])

            continue


        block = (blocks[block_idx][0], blocks[block_idx][1])
        for x, y in nbr_block_offsets:
            nbr_block = (block[0] + x, block[1] + y)
            if nbr_block not in block_to_block_idx_map:
                continue

            nbr_block_idx = block_to_block_idx_map[nbr_block]
            if block_point_counts[nbr_block_idx] < block_point_count_threshold:
                continue


            #print(block_idx, nbr_block_idx, block_point_counts[nbr_block_idx])

            block_point_indices[nbr_block_idx] = np.concatenate(
                [block_point_indices[nbr_block_idx], block_point_indices[block_idx]], axis=-1)
            block_point_indices[block_idx] = np.array([], dtype=np.int)
            block_merge_count = block_merge_count + 1
            break
    #print('{}-{} of {} blocks are merged.'.format(datetime.now(), block_merge_count, blocks.shape[0]))

    idx_last_non_empty_block = 0
    for block_idx in reversed(range(blocks.shape[0])):
        if block_point_indices[block_idx].shape[0] != 0:
            idx_last_non_empty_block = block_idx
            break

    # uniformly sample each block
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        #print(block_idx, point_indices.shape)
        block_points = xyz[point_indices]
        block_min = np.amin(block_points, axis=0, keepdims=True)
        xyz_grids = np.floor((block_points - block_min) / grid_size).astype(np.int)
        grids, point_grid_indices, grid_point_counts = np.unique(xyz_grids, return_inverse=True,
                                                                 return_counts=True, axis=0)
        grid_point_indices = np.split(np.argsort(point_grid_indices), np.cumsum(grid_point_counts[:-1]))
        grid_point_count_avg = int(np.average(grid_point_counts))
        point_indices_repeated = []
        for grid_idx in range(grids.shape[0]):
            point_indices_in_block = grid_point_indices[grid_idx]
            repeat_num = math.ceil(grid_point_count_avg / point_indices_in_block.shape[0])
            if repeat_num > 1:
                point_indices_in_block = np.repeat(point_indices_in_block, repeat_num)
                np.random.shuffle(point_indices_in_block)
                point_indices_in_block = point_indices_in_block[:grid_point_count_avg]
            point_indices_repeated.extend(list(point_indices[point_indices_in_block]))
        block_point_indices[block_idx] = np.array(point_indices_repeated)
        block_point_counts[block_idx] = len(point_indices_repeated)

    idx = 0
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        block_point_num = point_indices.shape[0]
        block_split_num = int(math.ceil(block_point_num * 1.0 / max_point_num))
        point_num_avg = int(math.ceil(block_point_num * 1.0 / block_split_num))
        point_nums = [point_num_avg] * block_split_num
        point_nums[-1] = block_point_num - (point_num_avg * (block_split_num - 1))
        starts = [0] + list(np.cumsum(point_nums))

        np.random.shuffle(point_indices)
        block_points = xyz[point_indices]


        block_min = np.amin(block_points, axis=0, keepdims=True)
        block_max = np.amax(block_points, axis=0, keepdims=True)
        #block_center = (block_min + block_max) / 2
        #block_center[0][-1] = block_min[0][-1]
        #block_points = block_points - block_center  # align to block bottom center
        x, y, z = np.split(block_points, (1, 2), axis=-1)

#         block_xzyrgbi = np.concatenate([x, z, y, i[point_indices]], axis=-1)
        block_xzyrgbi = np.concatenate([x, y, z], axis=-1)
        block_labels = labels[point_indices]

        for block_split_idx in range(block_split_num):
            start = starts[block_split_idx]
            point_num = point_nums[block_split_idx]
            #print(block_split_num, block_split_idx, point_num )



            end = start + point_num
            idx_in_batch = idx % batch_size
            data[idx_in_batch, 0:point_num, ...] = block_xzyrgbi[start:end, :]
            data_num[idx_in_batch] = point_num
            
            label_seg[idx_in_batch, 0:point_num] = block_labels[start:end]
            
            indices_split_to_full[idx_in_batch, 0:point_num] = point_indices[start:end]

            #print("indices_split_to_full", idx_in_batch, point_num, indices_split_to_full)

            if  (block_idx == idx_last_non_empty_block and block_split_idx == block_split_num - 1): #Last iteration

                item_num = idx_in_batch + 1
                
            idx = idx + 1
            
    return label_length, data, data_num, label_seg, indices_split_to_full, item_num, all_label_pred, indices_for_prediction


In [85]:
import math 

object_dict = {
    'BICYCLE' : 3,
    'PEDESTRIAN': 1,
    'ON_ROAD_OBSTACLE': 4,
    'VEHICLE':2
}


max_point_num = args.max_point_num

batch_size = 2048
data = np.zeros((batch_size, max_point_num, 5))
data_num = np.zeros((batch_size), dtype=np.int32)
label = np.zeros((batch_size), dtype=np.int32)
label_seg = np.zeros((batch_size, max_point_num), dtype=np.int32)
indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)

data_all= list()
data_num_all= list()
label_all= list()
label_seg_all= list()
indices_all= list()

id_h5 = 0
h5_batches = 200

for data_idx, dataset_file in enumerate(tqdm(sorted(lidar_pathlist))):
    
    # Get lidar points
    actual_index = actual_idx_list[data_idx]
    data_points = PyntCloud.from_file(dataset_file)
    pts_lidar = data_points
    
    folder = actual_index[0]
    dataset = int(actual_index[1])
    
    ########################################################
    log, idx = actual_index[0], int(actual_index[1])
    lidar_file = dataset_file

    argoverse_data = data_loader.get(log)
    city_name = argoverse_data.city_name
    
    log_dataidx_list = log_to_count_map[log]
    log_data_idx = log_dataidx_list.tolist().index(data_idx)
    
    lidar_pts = argoverse_data.get_lidar(log_data_idx)
    city_to_egovehicle_se3 = argoverse_data.get_pose(log_data_idx)
    roi_area_pts = city_to_egovehicle_se3.transform_point_cloud(lidar_pts) # more to city CS
    roi_area_pts = am.remove_non_roi_points(roi_area_pts, city_name) # remove outside roi points
    roi_area_pts = am.remove_ground_surface(roi_area_pts, city_name) # remove ground  
    roi_area_pts = city_to_egovehicle_se3.inverse_transform_point_cloud(
        roi_area_pts
    )# Back to lidar cs

    x = np.array(roi_area_pts[:,0])[:, np.newaxis]
    y = np.array(roi_area_pts[:,1])[:, np.newaxis]
    z = np.array(roi_area_pts[:,2])[:, np.newaxis]
    pts_lidar = np.concatenate([x,y,z], axis = 1)
    
#     if ground_removal: 
#         pts_lidar = gs.ground_segmentation(pts_lidar)
    ###########################################################
#     pts_lidar = np.dot(argo_to_kitti,pts_lidar.T).T
    
    # Get objects
    label_file = label_pathlist[data_idx]
    objects = get_objects_from_label(label_file)
    
    bounding_box = list()
    croppedpc = list()
    label_indices = list()
    instance_pts = list()
    instance_indices = list()
    class_labels = list()
    
#     label_mask_seg = np.zeros(len(pts_lidar))
#     label_cls = np.zeros(label_mask_seg[-1], dtype = int ) 
#     label_cls[label_mask_seg] = 1 # feature 1
#     label_pts = list(map(list,set(tuple(i) for i in croppedpc)))
#     label_num = len(label_pts) # feature 2
#     data_num = len(pts_lidar)
    
    labels = np.zeros(pts_lidar.shape[0])
    rgb = np.zeros(pts_lidar.shape[0])
    i = np.zeros(pts_lidar.shape[0])
    bounding_box1 = []
    bounding_box2 = []
    
    # Get labels
    for each in objects:
        bounding_box1 = createBBox(bounding_box1,*getCorners_argo(each.h,each.w,each.l, *np.dot(each.pos, argo_to_kitti), each.rot_mat_argo,rotation=False),color="yellow")
        object_id = object_dict[each.cls_type]
        
        each.pos = np.dot(each.pos, argo_to_kitti)
        each.l = each.l + 0.1* each.l
        each.w = each.w + 0.1* each.w
        each.h = each.h + 0.1* each.h
        out = filter_pointcloud(each, np.copy(pts_lidar))
        
#         croppedpc.extend(out[1].tolist() if type(out[1]) == np.ndarray else out[1])
        label_indices.extend(out[0].tolist() if type(out[0]) == np.ndarray else out[0])
        class_labels.extend( np.zeros(len(out[0]), dtype=int) + object_id )
        labels[label_indices] = object_id
        rgb[label_indices] = len(out[0])
        i[label_indices] = object_id
        
        pc_trans = out[2]
        bbox = out[3]
        bounding_box2 = createBBox(bounding_box2,*getCorners_argo(each.h,each.w,each.l, *each.pos, each.rot_mat_argo,rotation=False),color="yellow")
        cropped_pc = out[1]
        
#         break
        
    xyzirgb = pts_lidar

    data_full = np.hstack((pts_lidar, rgb[:,np.newaxis]))

    label_length, data, data_num, label_seg, indices_split_to_full, item_num, all_label_pred, indices_for_prediction = data_preprocessing(data_full, labels, max_point_num)

    data_data =data[0:item_num, ...].astype(np.float32) 
    data_num =data_num[0:item_num, ...] 
    indices_split_to_full = indices_split_to_full[0:item_num]
    label_seg_data = label_seg[0:item_num]
    batch_num = data.shape[0]

    data_data = np.concatenate(data_data[0:6, :, :3])
    label_data = np.concatenate(label_seg_data[0:6, :])
        
    break


In [16]:
# Vizualize actual pointcloud with boxes

new_pts = pd.DataFrame(np.array(pts_lidar)[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box1)

/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/pythreejs/traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.9598288628764933, 47.76553760943052, 21.76…

In [17]:
# Vizulaized cropped pointcloud with mod-boxes 

new_pts = pd.DataFrame(cropped_pc[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box2)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.170361952333853, -0.5527554605373521, 2.65…

In [83]:
# vizualize segmented points

data_pts = pts_lidar
new_pts1 = pd.DataFrame(np.array(data_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts1["red"]  = 255
new_pts1["green"]  = 121
new_pts1["blue"]  = 121
labelled_pts = data_pts[label_indices]
new_pts2 = pd.DataFrame(np.array(labelled_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts2["red"]  = 255
new_pts2["green"]  = 255
new_pts2["blue"]  = 255
cloud = PyntCloud(new_pts1.append(new_pts2))
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box1)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.6794755892224524, 47.725793096709424, 21.4…

In [86]:
# enlarged bboxes

# vizualize segmented points

data_pts = pts_lidar
new_pts1 = pd.DataFrame(np.array(data_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts1["red"]  = 255
new_pts1["green"]  = 121
new_pts1["blue"]  = 121
labelled_pts = data_pts[label_indices]
new_pts2 = pd.DataFrame(np.array(labelled_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts2["red"]  = 255
new_pts2["green"]  = 255
new_pts2["blue"]  = 255
cloud = PyntCloud(new_pts1.append(new_pts2))
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box2)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.6794755892224524, 47.725793096709424, 21.4…

In [32]:
# vizualize processed points

data_pts = data_data
new_pts1 = pd.DataFrame(np.array(data_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts1["red"]  = 255
new_pts1["green"]  = 121
new_pts1["blue"]  = 121
labelled_pts = data_pts[label_data == 2]
new_pts2 = pd.DataFrame(np.array(labelled_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts2["red"]  = 255
new_pts2["green"]  = 255
new_pts2["blue"]  = 255
cloud = PyntCloud(new_pts1.append(new_pts2))
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box1)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(2.688401460647583, 48.19766020774841, 22.073…

In [24]:
# vizualize transformed points

new_pts = pd.DataFrame(np.array(pc_trans)[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box1)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(4.808792027944373, 206.71896873160773, 20.62…

In [71]:
path_h5 = os.path.join("/home/kartik/DL_model/PointCNN/data/Argo_h5/train/train_full_0.h5")
data = h5py.File(path_h5)
h5_data = data['data'][:,:,0:3].astype(np.float32)
h5_label = data['label_seg'][...].astype(np.int64)

h5_data = np.concatenate(h5_data[0:6, :, :3])
h5_label = np.concatenate(h5_label[0:6, :])
        

In [60]:
# vizualize h5 points

data_pts = h5_data
new_pts1 = pd.DataFrame(np.array(data_pts)[:, 0:3], columns=['x', 'z', 'y'])
new_pts1["red"]  = 255
new_pts1["green"]  = 121
new_pts1["blue"]  = 121
labelled_pts = data_pts[h5_label == 8]
new_pts2 = pd.DataFrame(np.array(labelled_pts)[:, 0:3], columns=['x','z', 'y'])
new_pts2["red"]  = 255
new_pts2["green"]  = 255
new_pts2["blue"]  = 255
cloud = PyntCloud(new_pts1.append(new_pts2))
cloud.plot(initial_point_size=0.02, polylines= lines+bounding_box1)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(2.3492038249969482, 48.20943880081177, 21.81…

In [69]:
label_indices.__len__()

9305

In [70]:
sum(label_data == 2)

10713

In [73]:
sum(h5_label == 8)

10715

In [63]:
pts_lidar.shape[0]

34190

In [67]:
data_data.shape

(49152, 3)

In [68]:
np.arange(len(data_pts))[labels == 2].shape

IndexError: boolean index did not match indexed array along dimension 0; dimension is 49152 but corresponding boolean dimension is 34190

In [53]:
np.arange(len(data_pts))[label_data == 2].shape

(10711,)

In [72]:
np.arange(len(data_pts))[h5_label == 8].shape

(10715,)

### H5 file make final

In [352]:
import math 

object_dict = {
    'BICYCLE' : 4,
    'PEDESTRIAN': 1,
    'ON_ROAD_OBSTACLE': 5,
    'VEHICLE':2,
    'LARGE_VEHICLE':3
}


def compute_dataset( file_paths, h5_batches):
    
    p = current_process()
    print('process counter:', p._identity[0]-1, " till: ", (p._identity[0]-1)*h5_batches,  'pid:', os.getpid())
    process_idx = p._identity[0]
    
    max_point_num = 8192
    batch_size = 2048

    data_all= list()
    data_num_all= list()
    label_all= list()
    label_seg_all= list()
    indices_all= list()
    
    id_h5 = (p._identity[0]-1)

    for idx, dataset_file in enumerate(tqdm(sorted(file_paths))):

        data_idx = (p._identity[0]-1)*h5_batches + idx
        
        # Get lidar points
        actual_index = actual_idx_list[data_idx]
        data_points = PyntCloud.from_file(dataset_file)
        pts_lidar = data_points

        folder = actual_index[0]
        dataset = int(actual_index[1])

        log, idx = actual_index[0], int(actual_index[1])
        lidar_file = dataset_file

        argoverse_data = data_loader.get(log)
        city_name = argoverse_data.city_name

        log_dataidx_list = log_to_count_map[log]
        log_data_idx = log_dataidx_list.tolist().index(data_idx)

        lidar_pts = argoverse_data.get_lidar(log_data_idx)
        city_to_egovehicle_se3 = argoverse_data.get_pose(log_data_idx)
        roi_area_pts = city_to_egovehicle_se3.transform_point_cloud(lidar_pts) # more to city CS
        roi_area_pts = am.remove_non_roi_points(roi_area_pts, city_name) # remove outside roi points
        roi_area_pts = am.remove_ground_surface(roi_area_pts, city_name) # remove ground  
        roi_area_pts = city_to_egovehicle_se3.inverse_transform_point_cloud(
            roi_area_pts
        )# Back to lidar cs

        x = np.array(roi_area_pts[:,0])[:, np.newaxis]
        y = np.array(roi_area_pts[:,1])[:, np.newaxis]
        z = np.array(roi_area_pts[:,2])[:, np.newaxis]
        pts_lidar = np.concatenate([x,y,z], axis = 1)

        # Get objects
        label_file = label_pathlist[data_idx]
        objects = get_objects_from_label(label_file)

        label_indices = list()
        labels = np.zeros(pts_lidar.shape[0])
        rgb = np.zeros(pts_lidar.shape[0])
        i = np.zeros(pts_lidar.shape[0])

        # Get labels
        for each in objects:
            object_id = object_dict[each.cls_type]

            each.pos = np.dot(each.pos, argo_to_kitti)
            out = filter_pointcloud(each, np.copy(pts_lidar))

            label_indices.extend(out[0].tolist() if type(out[0]) == np.ndarray else out[0])
            labels[label_indices] = object_id
            rgb[label_indices] = len(out[0])
            i[label_indices] = object_id

        xyzirgb = pts_lidar

        data_full = np.hstack((pts_lidar, rgb[:,np.newaxis]))

        label_length, data, data_num, label_seg, indices_split_to_full, item_num, all_label_pred, indices_for_prediction = data_preprocessing(data_full, labels, max_point_num)

        data_data =data[0:item_num, ...].astype(np.float32) 
        data_num =data_num[0:item_num, ...] 
        indices_split_to_full = indices_split_to_full[0:item_num]
        label_seg_data = label_seg[0:item_num]
        batch_num = data.shape[0]

        data_all.append(data_data)
        data_num_all.append(data_num)
        label_all.append(item_num*(data_idx+1))
        label_seg_all.append(label_seg_data)
        indices_all.append(indices_split_to_full)

    data_data = np.concatenate(data_all)
    data_num_all = np.concatenate(data_num_all)
    label_seg_all = np.concatenate(label_seg_all)
    indices_all = np.concatenate(indices_all)

    # filename_h5 = os.path.join(save_h5_root , folders[0], folders[0]+ '_%s_%d.h5' % ("full", data_idx))
    folder_name = folders[0][len(os.path.dirname(folders[0]))+1:]
    filename_h5 = os.path.join(cwd, "data", "Argo_h5", folder_name, folder_name+ '_%s_%d.h5' % ("full", id_h5))
    print("Saved to {} ".format(data_idx), filename_h5)

    file = h5py.File(filename_h5, 'w')
    file.create_dataset('data', data=data_data)
    file.create_dataset('data_num', data=data_num_all)
    file.create_dataset('label', data=label_all)
    file.create_dataset('label_seg', data=label_seg_all)
    file.create_dataset('indices_split_to_full', data=indices_all)
    file.close()

    data_all= list()
    data_num_all= list()
    label_all= list()
    label_seg_all= list()
    indices_all= list()


In [353]:
import math 
from multiprocessing import current_process, Pool
import multiprocessing

object_dict = {
    'BICYCLE' : 4,
    'PEDESTRIAN': 1,
    'ON_ROAD_OBSTACLE': 5,
    'VEHICLE':2,
    'LARGE_VEHICLE':3
}


max_point_num = args.max_point_num

batch_size = 2048

data_all= list()
data_num_all= list()
label_all= list()
label_seg_all= list()
indices_all= list()

id_h5 = 0
h5_batch_size = 10

# create seperate processing pathlists
indice = np.arange(lidar_pathlist.__len__())[::h5_batch_size]
process_batches = int(lidar_pathlist.__len__()/h5_batch_size)
process_path_list = [sorted(lidar_pathlist)[i:i+h5_batch_size] for i in range(process_batches)]

print("Total process counter runs: ", process_batches)

with multiprocessing.Pool(processes=20) as pool:
    ret_list = pool.starmap(compute_dataset, [(each_list, h5_batch_size) for each_list in process_path_list])
    

Total process counter runs:  1312
process counter: 46  till:  460 pid: 20681
process counter: 51  till:  510 pid: 20686
process counter: 50  till:  500 pid: 20685
process counter: 48  till:  480 pid: 20683
process counter: 49  till:  490 pid: 20684
process counter: 53  till:  530 pid: 20688
process counter: 57  till:  570 pid: 20692
process counter: 56  till:  560 pid: 20691
process counter: 47  till:  470 pid: 20682
process counter: 52  till:  520 pid: 20687
process counter: 60  till:  600 pid: 20695
process counter: 55  till:  550 pid: 20690
process counter: 61  till:  610 pid: 20696
process counter: 62  till:  620 pid: 20697
process counter: 63  till:  630 pid: 20698
process counter: 64  till:  640 pid: 20699
process counter: 58  till:  580 pid: 20693
process counter: 65  till:  650 pid: 20700
process counter: 59  till:  590 pid: 20694
process counter: 54  till:  540 pid: 20689


Process ForkPoolWorker-56:
Process ForkPoolWorker-66:
Process ForkPoolWorker-48:
Process ForkPoolWorker-51:
Process ForkPoolWorker-61:
Process ForkPoolWorker-57:
Process ForkPoolWorker-59:
Process ForkPoolWorker-64:
Process ForkPoolWorker-52:
Process ForkPoolWorker-49:
Process ForkPoolWorker-53:
Process ForkPoolWorker-50:
Process ForkPoolWorker-47:
Process ForkPoolWorker-65:
Process ForkPoolWorker-62:
Process ForkPoolWorker-63:
Process ForkPoolWorker-54:
Process ForkPoolWorker-55:
Process ForkPoolWorker-58:
Process ForkPoolWorker-60:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rece

  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/multiprocessing/pool.py", line 47, in starmapst

  File "<ipython-input-352-b07c3c9a9a49>", line 29, in compute_dataset
    for idx, dataset_file in enumerate(tqdm(sorted(file_paths))):
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/__init__.py", line 28, in tqdm_notebook
    return _tqdm_notebook(*args, **kwargs)
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/__init__.py", line 28, in tqdm_notebook
    return _tqdm_notebook(*args, **kwargs)
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/__init__.py", line 28, in tqdm_notebook
    return _tqdm_notebook(*args, **kwargs)
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/__init__.py", line 28, in tqdm_notebook
    return _tqdm_notebook(*args, **kwargs)
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/__init__.py", line 28, in tqdm_notebook
    return _tqdm_notebook(*args, **kwargs)
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packag

  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 97, in __enter__
    self.acquire()
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 90, in acquire
    lock.acquire()
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 90, in acquire
    lock.acquire()
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 90, in acquire
    lock.acquire()
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 90, in acquire
    lock.acquire()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 90, in acquire
    lock.acquire()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/kartik/anaconda3/envs/sane35/lib/python3.6/site-packages/tqdm/std.py", line 97, in __enter__
    self.acquire()
  File "/home/kartik

KeyboardInterrupt: 

In [ ]:

## Old script

import math 
from multiprocessing import current_process, Pool

object_dict = {
    'BICYCLE' : 4,
    'PEDESTRIAN': 1,
    'ON_ROAD_OBSTACLE': 5,
    'VEHICLE':2,
    'LARGE_VEHICLE':3
}


max_point_num = args.max_point_num

batch_size = 2048

data_all= list()
data_num_all= list()
label_all= list()
label_seg_all= list()
indices_all= list()

id_h5 = 0
h5_batches = 5

with multiprocessing.Pool(processes=3) as pool:
    p.starmap(compute_dataset, [sorted(lidar_pathlist)])
    
for data_idx, dataset_file in enumerate(tqdm(sorted(lidar_pathlist))):
    
    # Get lidar points
    actual_index = actual_idx_list[data_idx]
    data_points = PyntCloud.from_file(dataset_file)
    pts_lidar = data_points
    
    folder = actual_index[0]
    dataset = int(actual_index[1])
    
    log, idx = actual_index[0], int(actual_index[1])
    lidar_file = dataset_file

    argoverse_data = data_loader.get(log)
    city_name = argoverse_data.city_name
    
    log_dataidx_list = log_to_count_map[log]
    log_data_idx = log_dataidx_list.tolist().index(data_idx)
    
    lidar_pts = argoverse_data.get_lidar(log_data_idx)
    city_to_egovehicle_se3 = argoverse_data.get_pose(log_data_idx)
    roi_area_pts = city_to_egovehicle_se3.transform_point_cloud(lidar_pts) # more to city CS
    roi_area_pts = am.remove_non_roi_points(roi_area_pts, city_name) # remove outside roi points
    roi_area_pts = am.remove_ground_surface(roi_area_pts, city_name) # remove ground  
    roi_area_pts = city_to_egovehicle_se3.inverse_transform_point_cloud(
        roi_area_pts
    )# Back to lidar cs

    x = np.array(roi_area_pts[:,0])[:, np.newaxis]
    y = np.array(roi_area_pts[:,1])[:, np.newaxis]
    z = np.array(roi_area_pts[:,2])[:, np.newaxis]
    pts_lidar = np.concatenate([x,y,z], axis = 1)
    
    # Get objects
    label_file = label_pathlist[data_idx]
    objects = get_objects_from_label(label_file)
    
    label_indices = list()
    labels = np.zeros(pts_lidar.shape[0])
    rgb = np.zeros(pts_lidar.shape[0])
    i = np.zeros(pts_lidar.shape[0])
    
    # Get labels
    for each in objects:
        object_id = object_dict[each.cls_type]
        
        each.pos = np.dot(each.pos, argo_to_kitti)
        out = filter_pointcloud(each, np.copy(pts_lidar))
        
        label_indices.extend(out[0].tolist() if type(out[0]) == np.ndarray else out[0])
        labels[label_indices] = object_id
        rgb[label_indices] = len(out[0])
        i[label_indices] = object_id

    xyzirgb = pts_lidar
    
    data_full = np.hstack((pts_lidar, rgb[:,np.newaxis]))
    
    label_length, data, data_num, label_seg, indices_split_to_full, item_num, all_label_pred, indices_for_prediction = data_preprocessing(data_full, labels, max_point_num)

    data_data =data[0:item_num, ...].astype(np.float32) 
    data_num =data_num[0:item_num, ...] 
    indices_split_to_full = indices_split_to_full[0:item_num]
    label_seg_data = label_seg[0:item_num]
    batch_num = data.shape[0]
    
    data_all.append(data_data)
    data_num_all.append(data_num)
    label_all.append(item_num*(data_idx+1))
    label_seg_all.append(label_seg_data)
    indices_all.append(indices_split_to_full)

    if data_idx%h5_batches == 0 and data_idx != 0:

        data_data = np.concatenate(data_all)
        data_num_all = np.concatenate(data_num_all)
        label_seg_all = np.concatenate(label_seg_all)
        indices_all = np.concatenate(indices_all)

        # filename_h5 = os.path.join(save_h5_root , folders[0], folders[0]+ '_%s_%d.h5' % ("full", data_idx))
        folder_name = folders[0][len(os.path.dirname(folders[0]))+1:]
        filename_h5 = os.path.join(cwd, "data", "Argo_h5", folder_name, folder_name+ '_%s_%d.h5' % ("full", id_h5))
        print("Saved to {} ".format(data_idx), filename_h5)

        file = h5py.File(filename_h5, 'w')
        file.create_dataset('data', data=data_data)
        file.create_dataset('data_num', data=data_num_all)
        file.create_dataset('label', data=label_all)
        file.create_dataset('label_seg', data=label_seg_all)
        file.create_dataset('indices_split_to_full', data=indices_all)
        file.close()
        
        data_all= list()
        data_num_all= list()
        label_all= list()
        label_seg_all= list()
        indices_all= list()
        
        id_h5+=1

In [328]:
np.concatenate(data_all).shape

(36, 8192, 3)

In [308]:
all_keys = list()
for data_idx, dataset_file in enumerate(tqdm(sorted(lidar_pathlist))):
    objects = get_objects_from_label(label_file)
    for each in objects:
        all_keys.append(each.cls_type)

In [309]:
np.unique(all_keys)

array(['LARGE_VEHICLE', 'ON_ROAD_OBSTACLE', 'PEDESTRIAN', 'VEHICLE'],
      dtype='<U16')

### Check data

In [29]:
file_test = os.path.join(cwd, "data", "Argo_h5", "train", "train_full_19.h5")
data = h5py.File(file_test)
data.keys()
# file.close()

<KeysViewHDF5 ['data', 'data_num', 'indices_split_to_full', 'label', 'label_seg']>

In [30]:
data['data_num'].shape, data['data'].shape, data['indices_split_to_full'].shape, data['label'].shape, data['label_seg'].shape

((2610,), (2610, 8192, 3), (2610, 8192), (250,), (2610, 8192))

In [34]:
points = []
labels = []
point_nums = []
labels_seg = []
indices_split_to_full = []

points.append(data['data'][:,:,0:4].astype(np.float32))
labels.append(data['label'][...].astype(np.int64))
point_nums.append(data['data_num'][...].astype(np.int32))
labels_seg.append(data['label_seg'][...].astype(np.int64))

In [69]:
data_pts = np.concatenate(points[0][0:6, :, :3])
label_idx = np.concatenate(labels_seg[0][0:6, :])
new_pts1 = pd.DataFrame(np.array(data_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts1["red"]  = 255
new_pts1["green"]  = 121
new_pts1["blue"]  = 121
labelled_pts = data_pts[label_idx == 8]
new_pts2 = pd.DataFrame(np.array(labelled_pts)[:, 0:3], columns=['x', 'y', 'z'])
new_pts2["red"]  = 255
new_pts2["green"]  = 255
new_pts2["blue"]  = 255
cloud = PyntCloud(new_pts1.append(new_pts2))
cloud.plot(initial_point_size=0.02)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(4.580923080444336, 35.17377305030823, 171.79…

In [58]:
labelled_pts.shape, data_pts.shape

((409600, 3), (409600, 3))

### Before training

In [67]:
# Prepare inputs
print('{}-Preparing datasets...'.format(datetime.now()))
is_list_of_h5_list = not data_utils.is_h5_list(args['filelist'])

print("not h5")
filelist_train = args['filelist']
    
lidar_pathlist = list()
label_pathlist = list()

root_dir = os.path.join(cwd, "data", "Argo")
# root_dir = os.path.join("/home/safeai-gpu2/mnt/safeai-dataserver/argoverse-1.1/argoverse-tracking")
split = "train"
data_loader = ArgoverseTrackingLoader(os.path.join(root_dir,split))
log_list = data_loader.log_list
for log in log_list:
    lidar_pathlist.extend(data_loader.get(log).lidar_list)
    label_pathlist.extend(data_loader.get(log).label_list)
assert len(lidar_pathlist) == len(label_pathlist)
# am = ArgoverseMap()
calib_file = data_loader.calib_filename
num_sample = len(lidar_pathlist)
image_idx_list = np.arange(num_sample)
imageset_dir = os.path.join(root_dir,split)
splitname = lambda x: [x[len(imageset_dir+"/"):-4].split("/")[0], x[len(imageset_dir+"/"):-4].split("/")[2].split("_")[1]]
actual_idx_list = [splitname(each) for each in lidar_pathlist]
print("image list sample is: ", actual_idx_list[0])

data_train, _, data_num_train, label_train, _ = data_utils.load_seg2(filelist_train)
# data_val, _, data_num_val, label_val, _ = data_utils.load_seg(args.filelist_val)

# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])

num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_train.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))
batch_num_val = math.ceil(num_val / batch_size)
print('{}-{:d} testing batches per test.'.format(datetime.now(), batch_num_val))




2019-12-23 01:31:47.127580-Preparing datasets...
not h5


image list sample is:  ['3d20ae25-5b29-320d-8bae-f03e9dc177b9', '315975023020283000']


AttributeError: module 'data_utils' has no attribute 'load_seg2'

In [109]:
points = []
labels = []
point_nums = []
labels_seg = []
indices_split_to_full = []

folder = os.path.dirname(filelist_train)
print(filelist_train)
file = open(filelist_train)

for line_num in tqdm(range(len(filelist))):
    line = next(file)
    idx  = line[:-2].split(",")
    am = 0
    data = get_lidar(data_loader, am, lidar_pathlist, idx, line_num)
    points.append(data.astype(np.float32))

    label = get_label(label_pathlist, line_num)
    
    labels.append(data['label'][...].astype(np.int64))
    point_nums.append(data['data_num'][...].astype(np.int32))
    labels_seg.append(data['label_seg'][...].astype(np.int64))
    if 'indices_split_to_full' in data:
        indices_split_to_full.append(data['indices_split_to_full'][...].astype(np.int64))
file.close()
_point = np.concatenate(points, axis=0)

_ref_max = np.max(_point[:,:,3:4].flatten())
_dens_max = np.max(_point[:,:,4:5].flatten())

_point[:,:,3:4] = (  _point[:,:,3:4] / _ref_max ) - 0.5
_point[:,:,4:5] = (  _point[:,:,4:5] / _dens_max ) - 0.5

print("amax", np.amax(_point, axis=0, keepdims=True))
print("amin", np.amin(_point, axis=0, keepdims=True))

return (_point[:,:,[0,1,2,3, 4]],
        np.concatenate(labels, axis=0),
        np.concatenate(point_nums, axis=0),
        np.concatenate(labels_seg, axis=0),
        np.concatenate(indices_split_to_full, axis=0) if indices_split_to_full else None)

/home/kartik/DL_model/PointCNN/data/KITTI/ImageSets/train.txt


IndexError: too many indices for array

In [ ]:
def get_label(label_pathlist, idx):

    label_file = label_pathlist[idx]
    assert os.path.exists(label_file)

    return kitti_utils.get_objects_from_label(label_file)

def get_objects_from_label(label_file):
    # Opens a label file, and passes the object to Object3d object, Read the json GT labels
    
    f = open(label_file)
    label_data = json.load(f) 
    objects = [object3d.Object3d(data) for data in label_data]
    return objects

### Vizulaization helper

In [239]:
new_pts = pd.DataFrame(np.array(np.concatenate(data[7:10, :, :3], axis=0))[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02)


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0)…